In [6]:
import csv
import collections
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np 
from sklearn import tree
from sklearn import preprocessing

In [7]:
numCols = list()

def mode(num_list):
    # print("len of num_list", len(num_list))
    # print("num_list1", num_list)
    num_list = np.array(list(filter(lambda a: a != 'NA', num_list)))
    # print("num_list2", num_list)
    modev = collections.Counter(num_list).most_common(1)
    mode_val = modev[0][0]
    return mode_val

def prepData(trainData):
    # trainData.sort(key=lambda x:x[-1])
    for i in range(len(trainData[0])):
        for j in range(len(trainData)):
            if trainData[j][i] != 'NA' and str.isdigit(trainData[j][i]) :
                numCols.append(i)
                break
    # print("NUMCOLS")
    # print(numCols)
    for i in numCols:
        num_list = list()
        for j in range(len(trainData)):
            num_list.append(trainData[j][i])
        mode_val = mode(num_list)
        # print("mode",mode_val)
        for j in range(len(num_list)):
            if trainData[j][i] == 'NA':
                trainData[j][i] = mode_val
    return trainData

# Split a dataset based on an attribute and an attribute value
def splitDataset(split_index, split_value, dataset):
    left, right = dataset, dataset
    if split_index in numCols :
        a = dataset[:,split_index]
        b = a.astype(np.float)
        c = b<=float(split_value)
        d = c.astype(np.float)
        indexes = np.where(d == 1)[0]
        left = dataset[np.r_[indexes],:]
        indexes = np.where(d == 0)[0]
        right = dataset[np.r_[indexes],:]
    else :
        a = dataset[:,split_index]
        c = a==split_value
        d = c.astype(np.float)
        indexes = np.where(d == 1)[0]
        left = dataset[np.r_[indexes],:]
        indexes = np.where(d == 0)[0]
        right = dataset[np.r_[indexes],:]
    return left, right

# Calculate the Mean squared error for a splitNode dataset
def meanSquaredError(left,right):
    mse = 0.0
    sizeC = 0
    groups = np.array([left, right])
    for group in groups:
        size = float(len(group))
        sizeC += size
        # avoid divide by zero
        if size == 0:
            continue
        groupVal = group[:,-1]
        groupVal = groupVal.astype(np.float)
        a = np.full(len(groupVal),np.average(groupVal))
        mse += size*mean_squared_error(groupVal, a)
    if sizeC != 0:
        mse = mse/sizeC
    return mse

# Create a terminal node(predict price)
def terminalNode(groupVal):
    outcomes = groupVal[:,-1].astype(np.float)
    return float(np.sum(outcomes) / len(outcomes)) 

# Find split point of least MSE
def findSplit(dataset):
    #intializing
    split_index = 0
    split_value = np.unique(dataset[:,0])[0]
    split_left, split_right = splitDataset(0, split_value, dataset)
    split_mse = meanSquaredError(split_left, split_right)

    for index in range(len(dataset[0])-1):
        uniqValue = np.unique(dataset[:,index])
        for value in uniqValue:
            left, right = splitDataset(index, value, dataset)
            mse = meanSquaredError(left, right)
            if mse < split_mse:
                split_index, split_value, split_mse, split_left, split_right = index, value, mse, left, right
    return {'indexAttr':split_index, 'valueAttr':split_value, 'left':split_left, 'right':split_right}

# Make node terminal or split further
def splitNode(node, max_depth, min_size, depth):
    left, right = node['left'], node['right']

    # check if already terminal node
    if left is None or right is None:
        node['left'] = terminalNode(left + right)
        node['right'] = node['left']
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = terminalNode(left), terminalNode(right)
        return

    if len(left) <= min_size:
        node['left'] = terminalNode(left)
    else:
        node['left'] = findSplit(left)
        splitNode(node['left'], max_depth, min_size, depth+1)

    if len(right) <= min_size:
        node['right'] = terminalNode(right)
    else:
        node['right'] = findSplit(right)
        splitNode(node['right'], max_depth, min_size, depth+1)

# Build a decision tree
def buildTree(trainData, max_depth, min_size):
    root = findSplit(trainData)
    splitNode(root, max_depth, min_size, 1)
    return root

In [8]:
# Predict with a decision tree
def predict(node, row):
    if node['indexAttr'] in numCols :
        # print(row)
        # print(node['indexAttr'])
        if float(row[node['indexAttr']]) <= float(node['valueAttr']):
            if isinstance(node['left'], float):
                return node['left']
            else :
                return predict(node['left'], row)               
        else:
            if isinstance(node['right'], float):
                return node['right']
            else :
                return predict(node['right'], row)       
    else:
        if row[node['indexAttr']] == node['valueAttr']:
            if isinstance(node['right'], float):
                return node['right']
            else :
                return predict(node['right'], row)
        else:
            if isinstance(node['right'], float):
                return node['right']
            else :
                return predict(node['right'], row)

In [9]:
def train(trainFile, max_depth, min_size):
    with open(trainFile,'r') as f:
        reader  = csv.reader(f)
        trainData = np.array(list(reader))
        #remove id column
        trainData = np.delete(trainData, 0, 1)
        trainData = np.delete(trainData,0,0)
        trainData = prepData(trainData)
        validate = trainData[0:400,:]
        trainData = trainData[400:,:]
        tags = np.empty(len(validate), dtype = "float") 
        predictions = np.empty(len(validate), dtype = "float")
        root = buildTree(trainData, max_depth, min_size)
        for index in range(len(validate)):
            # print("index ",index)
            test_row = validate[index]
            prediction = predict(root, test_row)
            tags[index] = float(test_row[-1])
            predictions[index] = prediction
        return tags, predictions

In [10]:
def evaluate(tags, predictions):
    print("Mean Absolute Error", mean_absolute_error(tags, predictions))
    print("Mean Squared Error", mean_squared_error(tags, predictions))

Build and Validate decision tree with Max depth of branch = 5, Min Size of node = 5

In [11]:
tags, predictions = train("./Datasets/q3/train.csv",5,5)
evaluate(tags, predictions)

Mean Absolute Error 31338.641685301136
Mean Squared Error 2243741870.4169755


In [ ]:
Build and Validate decision tree with Max depth of branch = 8, Min Size of node = 8

In [10]:
tags, predictions = train("./Datasets/q3/train.csv",8,8)
evaluate(tags, predictions)

Mean Absolute Error 30434.225399228082
Mean Squared Error 2052471604.891035


In [ ]:
Build and Validate decision tree with Max depth of branch = 6, Min Size of node = 8

In [11]:
tags, predictions = train("./Datasets/q3/train.csv",6,8)
evaluate(tags, predictions)

Mean Absolute Error 31182.752416984746
Mean Squared Error 2081739092.3071074


The best case for the implemented Decision tree, with the least Mean Absolute Error and Mean Squared Error is at Max depth of branch = 8, Min Size of node = 8 

When the Data is normalized

In [33]:
def prepData(trainData):
    # trainData.sort(key=lambda x:x[-1])
    for i in range(len(trainData[0])):
        for j in range(len(trainData)):
            if trainData[j][i] != 'NA' and str.isdigit(trainData[j][i]) :
                numCols.append(i)
                break
    # print("NUMCOLS")
    # print(numCols)
    for i in numCols:
        num_list = list()
        for j in range(len(trainData)):
            num_list.append(trainData[j][i])
        mode_val = mode(num_list)
        # print("mode",mode_val)
        for j in range(len(num_list)):
            if trainData[j][i] == 'NA':
                trainData[j][i] = mode_val

        a = trainData[:,i].astype(np.float)
        a = a - np.mean(a)
        a = a / np.std(a)
        trainData[:,i] = a.astype(np.str)
    return trainData

Build and Validate decision tree with Max depth of branch = 5, Min Size of node = 5 (Data normalized)

In [34]:
tags, predictions = train("./Datasets/q3/train.csv",5,5)
evaluate(tags, predictions)

Mean Absolute Error 0.38101765860283593
Mean Squared Error 0.3316673818557357


Build and Validate decision tree with Max depth of branch = 8, Min Size of node = 8 (Data normalized)

In [35]:
tags, predictions = train("./Datasets/q3/train.csv",8,8)
evaluate(tags, predictions)

Mean Absolute Error 0.37058456613029833
Mean Squared Error 0.30336472470351533


Build and Validate decision tree with Max depth of branch = 7, Min Size of node = 8 (Data normalized)

In [46]:
tags, predictions = train("./Datasets/q3/train.csv",7,8)
evaluate(tags, predictions)

Mean Absolute Error 0.37995771406034945
Mean Squared Error 0.307614299940534


The best case for the implemented Decision tree (with Data normalized), with the least Mean Absolute Error and Mean Squared Error is at Max depth of branch = 8, Min Size of node = 8

Comparison with the inbuilt Decision tree of sklearn, mean choosing and median choosing for all test rows

In [43]:
#Inbuilt implementation of decision tree using sklearn
def inbuiltDecisionTree(trainFile):
    with open(trainFile,'r') as f:
        reader  = csv.reader(f)
        trainData = np.array(list(reader))
        #remove id column
        trainData = np.delete(trainData, 0, 1)
        trainData = np.delete(trainData,0,0)
        trainData = prepData(trainData)
        le = preprocessing.LabelEncoder()
        for col in range(len(trainData[0])):
            le.fit(trainData[:,col])
            trainData[:,col] = le.transform(trainData[:,col])
        validate = trainData[0:400,:]
        trainData = trainData[400:,:]
        tags = validate[:,-1].astype(np.float)
        predictions = np.empty(len(validate), dtype = "float")
        clf = tree.DecisionTreeRegressor()
        l = len(trainData)
        clf = clf.fit(trainData[:,0:l-1], trainData[:,-1])
        predictions = clf.predict(validate)
        return tags, predictions

#Mean choosing
def mean_classifier(trainFile):
    with open(trainFile,'r') as f:
        reader  = csv.reader(f)
        trainData = np.array(list(reader))
        #remove id column
        trainData = np.delete(trainData, 0, 1)
        trainData = np.delete(trainData,0,0)
        trainData = prepData(trainData)
        validate = trainData[0:400,:]
        trainData = trainData[400:,:]
        tags = validate[:,-1].astype(np.float)
        predictions = np.full(len(validate), np.mean(trainData[:,-1].astype(np.float)))
        return tags, predictions

#Median choosing
def median_classifier(trainFile):
    with open(trainFile,'r') as f:
        reader  = csv.reader(f)
        trainData = np.array(list(reader))
        #remove id column
        trainData = np.delete(trainData, 0, 1)
        trainData = np.delete(trainData,0,0)
        trainData = prepData(trainData)
        validate = trainData[0:400,:]
        trainData = trainData[400:,:]
        tags = validate[:,-1].astype(np.float)
        predictions = np.full(len(validate), np.median(trainData[:,-1].astype(np.float)))
        return tags, predictions

Inbuilt decision tree with Data Normalized

In [41]:
tags, predictions = inbuiltDecisionTree("./Datasets/q3/train.csv")
evaluate(tags, predictions)

Mean Absolute Error 1.39
Mean Squared Error 3.815


Mean choosing with Data normalized

In [44]:
tags, predictions = mean_classifier("./Datasets/q3/train.csv")
evaluate(tags, predictions)

Mean Absolute Error 0.75672820105992
Mean Squared Error 1.024717302166257


Median choosing with Data normalized

In [45]:
tags, predictions = median_classifier("./Datasets/q3/train.csv")
evaluate(tags, predictions)

Mean Absolute Error 0.71697263567595
Mean Squared Error 1.0564554556253556


Our decision tree is impemented using major libraries such as numpy, collections. The tree's training is done on part of the train.csv's data and its validation is done on remaining data. 
Our implementation is giving its best performance(least Mean Absolute Error and least Mean Squared Error) for Max depth of branch = 8, Min Size of node = 8. It is comparable with the Inbuilt decision tree from sklearn library. It does better than the baseline cases of mean choosing/ median choosing.  